In [73]:
import requests
import json
import pandas as pd

In [74]:
def extract_matches(year):
    # Extracting last matches date
    api_key = eval(open('config.txt').read())['auth']
    url = f'https://api.sportsdata.io/v3/nba/scores/json/Games/{year}?key={api_key}'
    response = requests.get(url).json()
    df = pd.DataFrame(response)
    df = pd.concat([df[df['HomeTeam'] == 'LAL'], df[df['AwayTeam'] == 'LAL']])
    df['DateTime'] = [pd.to_datetime(d).strftime(format='%Y-%m-%d') for d in df['DateTime']]
    return df
def extract_next_match():
    df = extract_matches(2023)
    # Extracting next match date
    df_ended = df[df['Status'] != 'Scheduled'].sort_values(by='DateTime', ascending=True).reset_index(drop=True)
    df_scheduled = df[df['Status'] == 'Scheduled'].sort_values(by='DateTime', ascending=True).reset_index(drop=True)
    next_match = df_scheduled.iloc[0]
    next_match_date = next_match['DateTime']
    home_team = next_match['HomeTeam']
    away_team = next_match['AwayTeam']
    # Extracting last two matches score
    if home_team == 'LAL':
        vs_team = away_team
    else:
        vs_team = home_team
    df_2022 = extract_matches(2022)
    last_matches = pd.concat([df_ended[df_ended['HomeTeam'] == vs_team], df_ended[df_ended['AwayTeam'] == vs_team], df_2022[df_2022['HomeTeam'] == vs_team], df_2022[df_2022['AwayTeam'] == vs_team]])
    last_matches = last_matches.sort_values(by='DateTime', ascending=True).reset_index(drop=True)
    last_matches = last_matches.iloc[-2:]
    display(last_matches[['DateTime', 'HomeTeam', 'AwayTeam', 'HomeTeamScore', 'AwayTeamScore']])

In [75]:
if __name__ == '__main__':
    extract_next_match()

,DateTime,HomeTeam,AwayTeam,HomeTeamScore,AwayTeamScore
0,2022-01-07,LAL,ATL,62.0,55.0
1,2022-01-30,ATL,LAL,60.0,56.0
